In [1]:
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

from IPython.display import clear_output

In [2]:
dir_img = '/home/ubuntu/T2_Sag_Mid/train_aug_data/elastic_imgs/'
dir_mask = '/home/ubuntu/T2_Sag_Mid/train_aug_data/elastic_masks/'

dir_checkpoint = 'ckpts_aug_dir/mid_elast_ckpts/'

try:
# Create target Directory
    os.mkdir(dir_checkpoint)
    print("Directory " , dir_checkpoint ,  " Created ") 
except FileExistsError:
    print("Directory " , dir_checkpoint ,  " already exists")

Directory  ckpts_aug_dir/mid_elast_ckpts/  already exists


In [3]:
def train_net(net,
              device,
              epochs=5,
              batch_size=1,
              lr=0.001,
              val_percent=0.1,
              save_cp=True,
              img_scale=0.5,
              epoch_bias = 0):

    dataset = BasicDataset(dir_img, dir_mask, img_scale)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 2 else 'max', patience=2)
    if net.n_classes > 2:
        print("Using CrossEntropyLoss")
        criterion = nn.CrossEntropyLoss()
    else:
        print("Using BCEWithLogitsLoss")
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                mask_type = torch.float32 if net.n_classes <= 2 else torch.long
                true_masks = true_masks.to(device=device, dtype=mask_type)

                masks_pred = net(imgs)
                
                loss = criterion(masks_pred, true_masks)
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                if global_step % (len(dataset) // (10 * batch_size)) == 0:
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)

                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                    if net.n_classes > 2:
                        logging.info('Validation cross entropy: {}'.format(val_score))
                        writer.add_scalar('Loss/test', val_score, global_step)
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val_score))
                        writer.add_scalar('Dice/test', val_score, global_step)

                    writer.add_images('images', imgs, global_step)
                    if net.n_classes == 1:
                        writer.add_images('masks/true', true_masks, global_step)
                        writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            if (epoch + epoch_bias + 1) % 50 == 0:
                torch.save(net.state_dict(),
                           dir_checkpoint + f'CP_epoch{epoch + epoch_bias + 1}.pth')
                logging.info(f'Checkpoint {epoch + epoch_bias + 1} saved !')

    writer.close()

In [4]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [5]:
net = UNet(n_channels=1, n_classes=2, bilinear=True)
logging.info(f'Network:\n'
             f'\t{net.n_channels} input channels\n'
             f'\t{net.n_classes} output channels (classes)\n'
             f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')

INFO: Network:
	1 input channels
	2 output channels (classes)
	Bilinear upscaling


In [6]:
n_classes = 2 
n_channels = 1 

In [ ]:
epochs = 300 
batchsize = 1

load = '/home/ubuntu/unet4/ckpts_dir/mid_ckpts/CP_epoch700.pth'
epoch_bias = 700

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)
        


INFO: Model loaded from /home/ubuntu/unet4/ckpts_dir/mid_ckpts/CP_epoch700.pth
INFO: Creating dataset with 3981 examples
INFO: Starting training:
        Epochs:          300
        Batch size:      1
        Learning rate:   0.001
        Training size:   3583
        Validation size: 398
        Checkpoints:     True
        Device:          cuda
        Images scaling:  0.5
    
Epoch 1/300:   0%|          | 0/3583 [00:00<?, ?img/s]

Using BCEWithLogitsLoss


Epoch 1/300:   8%|▊         | 277/3583 [00:21<04:13, 13.06img/s, loss (batch)=0.0238] 